In [488]:
import cv2
import torchvision
import torch
import torchvision.transforms as transforms
import PIL.Image as Image
import numpy as np

In [489]:
# Load the model
model = torch.load('model.pth')

In [490]:
# Define the classes
classes = ['combat', 'destroyed_buildings', 'fire', 'humanitarinaid', 'militaryvehicles']

In [491]:
# Load the image
img = cv2.imread('Images/final.jpg')

In [492]:
# Define the mean and std
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Define the image transforms
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [493]:
def classify(model, image_transforms, image, classes):
    model = model.eval()
    # Move model to the same device as the image tensor
    device = next(model.parameters()).device
    # Apply transformations and move image to the device
    image = image_transforms(image).float().to(device)
    image = image.unsqueeze(0)

    output = model(image)
    _, predicted = torch.max(output.data, 1)

    return classes[predicted.item()]

In [494]:
def draw_and_measure_boxes(img, threshold_value=200, box_color=(0, 255, 0), box_thickness=2, min_area=10000, max_area=13000):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY)[1]

    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bounding_boxes = []
    for cntr in contours:
        x, y, w, h = cv2.boundingRect(cntr)
        area = w * h

        if min_area <= area <= max_area:
            cv2.rectangle(img, (x, y), (x+w, y+h), box_color, box_thickness)
            bounding_boxes.append((x, y, w, h, area))
    
    for i, box in enumerate(bounding_boxes):
        x, y, w, h, area = box
        label = classify(model, image_transforms, Image.fromarray(cv2.cvtColor(img[y:y+h, x:x+w], cv2.COLOR_BGR2RGB)), classes)
        cv2.putText(img, label, (x + 5, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)
    return bounding_boxes, img

In [495]:
# Define the super-resolution model
sr = cv2.dnn_superres.DnnSuperResImpl_create()
path = "EDSR_x2.pb"
sr.readModel(path)
sr.setModel("edsr", 2)

In [496]:
# Draw and measure the bounding boxes
bounding_boxes, img_with_boxes = draw_and_measure_boxes(img)

In [497]:
# Display the image with bounding boxes
cv2.imshow('Image with Bounding Boxes', img_with_boxes)
cv2.waitKey(0)
cv2.destroyAllWindows()